In [5]:
import pandas as pd
import mysql

In [16]:
import mysql
from mysql.connector import MySQLConnection, Error
import pandas as pd
import time



df = pd.DataFrame()
def query_with_fetchall():
    try:
        connection = mysql.connector.connect(user='brett', 
                                             password='BrettTully#2023',
                                             host='r2klabs.com',
                                             database='sensor')
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM rpisense order by sensor_date DESC;")
        rows = cursor.fetchall()
        print('Total Row(s):', cursor.rowcount)
         #only print head
        title = [i[0] for i in cursor.description]
        print(title)
        #for row in rows:
         #   print(row)
        global df
        df = pd.DataFrame(rows)
        df['time'] = df[1].dt.time
#        df['time'] = time.strftime("%I:%M %p")
        df['weekday'] = df[1].dt.weekday
        df['date'] = df[1].dt.date
        

    except Error as e:
        print(e)

    finally:
        cursor.close()
        connection.close()

query_with_fetchall()
df

Total Row(s): 727312
['device', 'sensor_date', 'temp', 'humidity']


,0,1,2,3,time,weekday,date
0,P006B,2023-03-22 18:53:55,69.80,48.7,18:53:55,2,2023-03-22
1,P004,2023-03-22 18:53:55,67.82,49.4,18:53:55,2,2023-03-22
2,P002,2023-03-22 18:53:55,65.84,66.9,18:53:55,2,2023-03-22
3,P002,2023-03-22 18:53:53,65.84,66.9,18:53:53,2,2023-03-22
4,P004,2023-03-22 18:53:53,67.82,49.4,18:53:53,2,2023-03-22
...,...,...,...,...,...,...,...
727307,P002,2023-03-16 13:48:57,65.48,51.4,13:48:57,3,2023-03-16
727308,P002,2023-03-16 13:48:54,65.48,51.4,13:48:54,3,2023-03-16
727309,P002,2023-03-16 13:48:52,65.48,49.8,13:48:52,3,2023-03-16
727310,P002,2023-03-16 13:47:38,65.48,49.7,13:47:38,3,2023-03-16


In [8]:
from datetime import date
import calendar

for d in df[1]:
    df['weekday'] = d.weekday()
    df['date'] = d.date()
    df['time'] = d.time()
    
print(df)

KeyboardInterrupt: 

In [13]:
df['time'] = df[1].dt.time
df

,0,1,2,3,time
0,P006B,2023-03-22 18:48:11,69.62,48.6,18:48:11
1,P002,2023-03-22 18:48:11,65.84,67.0,18:48:11
2,P004,2023-03-22 18:48:10,67.82,49.4,18:48:10
3,P002,2023-03-22 18:48:09,65.84,67.0,18:48:09
4,P006B,2023-03-22 18:48:09,69.62,48.6,18:48:09
...,...,...,...,...,...
726785,P002,2023-03-16 13:48:57,65.48,51.4,13:48:57
726786,P002,2023-03-16 13:48:54,65.48,51.4,13:48:54
726787,P002,2023-03-16 13:48:52,65.48,49.8,13:48:52
726788,P002,2023-03-16 13:47:38,65.48,49.7,13:47:38


In [ ]:
import datetime

# Dictionary of occupied times for each classroom and day of the week
occupied_times = {
    "P002": {
        "Monday": ["08:00:00", "10:00:00", "12:00:00", "14:00:00", "16:00:00"],
        "Tuesday": ["09:00:00", "11:00:00", "13:00:00", "15:00:00", "17:00:00"],
        "Wednesday": ["10:00:00", "12:00:00", "14:00:00", "16:00:00", "18:00:00"],
        "Thursday": ["11:00:00", "13:00:00", "15:00:00", "17:00:00", "19:00:00"],
        "Friday": ["12:00:00", "14:00:00", "16:00:00", "18:00:00", "20:00:00"]
    },
    "P004": {
        "Monday": ["09:00:00", "11:00:00", "13:00:00", "15:00:00", "17:00:00"],
        "Tuesday": ["10:00:00", "12:00:00", "14:00:00", "16:00:00", "18:00:00"],
        "Wednesday": ["11:00:00", "13:00:00", "15:00:00", "17:00:00", "19:00:00"],
        "Thursday": ["12:00:00", "14:00:00", "16:00:00", "18:00:00", "20:00:00"],
        "Friday": ["13:00:00", "15:00:00", "17:00:00", "19:00:00", "21:00:00"]
    }
    "P004": {
        "Monday": ["09:00:00", "11:00:00", "13:00:00", "15:00:00", "17:00:00"],
        "Tuesday": ["10:00:00", "12:00:00", "14:00:00", "16:00:00", "18:00:00"],
        "Wednesday": ["11:00:00", "13:00:00", "15:00:00", "17:00:00", "19:00:00"],
        "Thursday": ["12:00:00", "14:00:00", "16:00:00", "18:00:00", "20:00:00"],
        "Friday": ["13:00:00", "15:00:00", "17:00:00", "19:00:00", "21:00:00"]
    }
}

# Get the current day of the week, time, and classroom number
now = datetime.datetime.now()
day_of_week = now.strftime("%A")
time = now.strftime("%H:%M:%S")
classroom_number = "101"  # Replace with the classroom number you want to check

# Check if the current time and day of the week is in the list of occupied times for the classroom
is_occupied = False
if classroom_number in occupied_times:
    if day_of_week in occupied_times[classroom_number]:
        is_occupied = time in occupied_times[classroom_number][day_of_week]

# Print the result
print(f"The classroom {classroom_number} is occupied:", is_occupied)

In [36]:
df = DataFrame()

NameError: name 'DataFrame' is not defined